In [115]:
import json
from collections import defaultdict
import random

In [100]:
with open('cor.json') as data:
    file = json.load(data)

In [101]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [102]:
import nltk 
from nltk import wordpunct_tokenize

In [103]:
dialogues = [] 
for d in file[:2]:
    samples = defaultdict(dict)
    result = d['completions'][0]['result']
    texts_without_labels = d['data']['text']
    for sample in result:
        speaker = texts_without_labels[int(sample['value']['start'])]['speaker']
        samples[sample['id']]['speaker'] = speaker
        samples[sample['id']]['text'] = sample['value']['text']
        samples[sample['id']]['start'] = int(sample['value']['start'])
        if 'paragraphlabels' in sample['value']:
            samples[sample['id']]['paragraphlabels'] = sample['value']['paragraphlabels'][0]
        if 'choices' in sample['value']:
            samples[sample['id']]['choices'] = sample['value']['choices'][0]
    
    sorted_samples = sorted([(samples[sample_id]['start'], sample_id) for sample_id in samples])
    texts = []
    labels = []
    speakers = []
    for _, sample_id in sorted_samples:
        if samples[sample_id]['text'] != 'PAUSE':
            texts.append(str(samples[sample_id]['text']).replace('\n', ''))
            speakers.append(samples[sample_id]['speaker'])
            paragraph_labels = samples[sample_id].get('paragraphlabels', '')
            choices = samples[sample_id].get('choices', '')
            labels.append(paragraph_labels + '.' + choices)
    dialogues.append((texts, labels, speakers))

In [104]:
train_data = dialogues[1][0]
test_data = dialogues[0][0]
all_data=train_data+test_data

In [105]:
train_labels = dialogues[1][1]
test_labels = dialogues[0][1]
all_labels=train_labels+test_labels

In [134]:
registers=[]
for i in zip(all_data, all_labels):
    if 'Register' in i[1]:
        if len(wordpunct_tokenize(i[0]))<3:
            if not 'Leah' in i[0]:
                if not 'shit' in i[0]:
                    registers.append(i[0].strip('.'))

In [138]:
registers=list(set(registers))

In [164]:
sustain_monitor=['You know?', 'Alright?','Yeah?','See?','Right?']

In [222]:
def generate_response(predicted_sf, previous_phrase, enable_repeats_register=False, user_name=''):
    if 'Register' in predicted_sf:
        response = random.choice(registers)
        if enable_repeats_register==True:
            response=word_tokenize(previous_phrase)[-1].capitalize()+'.'
    if 'Check' in predicted_sf:
        response=random.choice(track_check)
    if 'Confirm' in predicted_sf:
        response=confirm_response(previous_phrase)
    if 'Monitor' in predicted_sf:
        response=user_name+random.choice(sustain_monitor)  #можно добавить имя пользователя
    if 'Affirm' in predicted_sf:
        response=random.choice(reply_affirm+"Yeah,"+word_tokenize(previous_phrase)[-1])
    if 'Disawow' in predicted_sf:
        response=random.choice(reply_disawow)
    if 'Disagree' in predicted_sf:
        response=random.choice(reply_disagree)
    if 'Agree' in predicted_sf:
        response=random.choice(reply_agree)
    if 'Contradict' in predicted_sf:
        response=random.choice(reply_contradict)
    if 'Clarify' in predicted_sf:
        response=clarify_response(previous_phrase)        
    return response    

In [179]:
reply_agree=["Oh that's right. That's right.", "Yep.", "Right.", 'Sure', 'Indeed', 'I agree with you']

In [180]:
reply_disagree=['No', 'Hunhunh.', "I don't agree with you", "I disagree", "I do not think so", "I hardly think so", "I can't agree with you"]

In [181]:
reply_disawow=['I doubt it. I really do.', "I don't know.", "I'm not sure", 'Probably.', "I don't know if it's true"]

In [182]:
reply_acknowledge=['I knew that.','I know.', 'No doubts', 'I know what you meant.', 'Oh yeah.','I see']

In [183]:
reply_affirm=['Oh definitely.', 'Yeah.', 'Kind of.', 'Unhunh', 'Yeah I think so', 'Really.','Right.', "That's what it was."]

In [185]:
reply_contradict=['Oh definitely no', 'No', 'No way', 'Absolutely not', 'Not at all', 'Nope', 'Not really', 'Hardly']

In [220]:
def clarify_response(previous_phrase):
    doc=nlp(previous_phrase)
    poses=[]
    deps=[]
    for token in doc:
        poses.append(token.pos_)
        deps.append(token.dep_)
        if token.pos_=='NOUN' or token.pos_=='PROPN':
            clarify_noun=token.text
            next_sent='What '+clarify_noun+'?'
        elif token.dep_=='prep':
            prep=token.text
            next_sent=str(prep).capitalize() + ' what?' 
        elif poses[0]=='PROPN' or poses[0]=='PRON':
            if word_tokenize(previous_phrase)[0]=='I' or word_tokenize(previous_phrase)[0]=='We':
                first_pron='You'
                next_sent=first_pron + ' what?'
            else:
                next_sent=word_tokenize(previous_phrase)[0] + ' what?'
        else:
            next_sent='What?'
    return next_sent

In [186]:
def confirm_response(previous_phrase):
    track_confirm = ['Oh really?', ' Oh yeah?', 'Sure?', 'Are you sure?', 'Are you serious?','Yeah']
    if len(word_tokenize(previous_phrase))>5:
        next_sent=(word_tokenize(previous_phrase))[-1].capitalize()+'?'
    elif len(word_tokenize(previous_phrase))<4:
        if 'I' in previous_phrase:
            previous_phrase =re.sub('I','you', previous_phrase)
        next_sent=previous_phrase+'?'
    else:
        next_sent=random.choice(track_confirm)
    return next_sent

In [42]:
track_confirm=[' Oh really ?','Right ?', ' Okay ?']

In [141]:
track_check=['Pardon?', 'I beg your pardon ?', 'Mhm ?','Hm?','What do you mean?']